# Analyzing Various Business KPIs Using SQL

# Analyzing Varioud Business KPI Using SQL
UBer Eats와 비슷한 Food Delivery Business 에서 발생하는 비지니스 KPI들에 대해 SQL을 사용하여 도출하겠다.

<img width="559" alt="delivr_business" src="https://user-images.githubusercontent.com/50973416/59886734-22b6f200-93fb-11e9-85d5-e61467ba0884.png">


## 다음과 같은 주요 Business KPI를 살펴본다.

### Revenue, Cost, Profit

1. Revenue per customer
2. Revenue per week
3. Top meals by cost
4. Cost per month
5. Profit per eatery
6. Profit per month

### User-Centric KPIs

1. Registrations by month
2. Monthly Active Users(MAU)
3. Registrations running total
4. MAU per month
5. Monthly MAU Growth Rate
6. Order Growth Rate
7. Retention Rate

### ARPU, histograms and percentiles

1. Average revenue per user (ARPU)
2. ARPU per week
3. Average order per user
4. Histogram of revenue
5. Histogram of orders
6. Bucketing users by revenue
7. Bucketing users by orders
8. Revenue quartiles
9. Interquartile range

### Generating an reports

1. Rank users by their count of orders
2. Pivoting user revenues by month




### 비지니스 KPI 측정을 통해 비지니스의 성장성과 서비스 개선에 대한 방향을 도출하겠다.

### Revenue per Customer

user_id = 15 인 customer에 대한 revenue을 구하겠다.

In [1]:
sql ='''

SELECT 
SUM(meals.meal_price * orders.order_quantity) AS revenue
FROM meals
JOIN orders ON meals.meal_id = orders.meal_id
WHERE user_id = 15;

'''

### Revenue per Week

6월 동안의 revenue per week을 구하겠다.

In [2]:
sql = '''

SELECT 
DATE_TRUNC('week', o.order_date) :: DATE AS delivr_week,
SUM(m.meal_price * o.order_quantity) As revenue
FROM meals As m
JOIN orders As o ON m.meal_id = o.meal_id
WHERE o.order_date LIKE '2018-06%'
GROUP BY delivr_week
ORDER BY delivr_week ASC;

'''

### Top Meals by Cost
각각의 `meal_id`에 대하여 cost를 구하고, top 5 cost를 추출한다.

In [3]:
sql = '''

SELECT
    meals.meal_id,
    SUM(stock.stocked_quantity * meals.meal_cost) AS cost
FROM meals
JOIN stock ON meals.meal_id = stock.meal_id
GROUP BY meals.meal_id
ORDER BY cost DESC
LIMIT 5;

'''

### Cost per Month

매월 총 얼마의 비용이 발생하는 지 구하겠다.

In [4]:
sql = '''

SELECT
    DATE_TRUNC('month', stock.stocking_date) :: DATE AS delivr_month
    SUM(stock.stocked_quantity * meals.meal_cost) AS cost
FROM meals
JOIN stock ON meals.meal_id = stock.meal_id
GROUP BY delivr_month
ORDER BY delivr_month ASC;

'''

### Profit per Eatery

각각 Eatery에 따른 Profit을 구하겠다. 

In [5]:
sql = '''

WITH revenue AS(

SELECT 
    meals.eatery,
    SUM(meal_price * order_quantity) AS revenue
FROM meals
JOIN orders ON meals.meal_id = orders.meal_id
GROUP BY eatery
),

cost AS(
SELECT 
    meals.eatery,
    SUM(meals.meal_cost * stock.stocked_quantity) AS cost
FROM meals
JOIN stock ON meals.meal_id = stock.meal_id
GROUP BY eatery
)



SELECT 
    revenue.eatery,
    revenue.revenue - cost.cost AS profit
FROM revenue
JOIN cost ON revenue.eatery = cost.eatery
ORDER BY profit DESC;

'''

### Profit per Month

매월 수익을 구하겠다.

In [6]:
sql = '''


WITH revenue AS ( 
SELECT
    DATE_TRUNC('month', orders.order_date) :: DATE AS delivr_month,
    SUM(meals.meal_price * orders.order_quantity) AS revenue
FROM meals
JOIN orders ON meals.meal_id = orders.meal_id
GROUP BY delivr_month),

cost AS (
SELECT
    DATE_TRUNC('month', stock.stocking_date) :: DATE AS delivr_month,
    SUM(meals.meal_cost * stock.stocked_quantity) AS cost
FROM meals
JOIN stock ON meals.meal_id = stock.meal_id
GROUP BY delivr_month)


SELECT
    revenue.delivr_month,
    revenue.revenue - cost.cost AS profit
FROM revenue
JOIN cost ON revenue.delivr_month = cost.delivr_month
ORDER BY revenue.delivr_month ASC;

'''

### Calculate Registrations Date
여기서는 첫 order를 한 날을 첫 registration으로 설정한다. 일반적으로는 따로 registration date 칼럼이 존재한다.

In [7]:
sql = '''

SELECT
    user_id,
    MIN(order_date) AS reg_date
FROM orders
GROUP BY user_id
ORDER BY user_id ASC;

'''

### Montly Active Users(MAU)

In [8]:
sql = '''

SELECT
    DATE_TRUNC('month',order_date) :: DATE AS delivr_month,
    COUNT(DISTINCT user_id) AS mau
FROM orders
GROUP BY delivr_month
ORDER BY delivr_month ASC;

'''

### Registrations Running Total
매달마다 누적 Resgistrations 수를 구하겠다.

In [9]:
sql = '''

WITH reg_dates AS (
SELECT
    user_id,
    MIN(order_date) AS reg_date
FROM orders
GROUP BY user_id)



SELECT
  DATE_TRUNC('month',reg_date) :: DATE AS delivr_month,
  COUNT(user_id) AS regs
FROM reg_dates
GROUP BY delivr_month
ORDER BY delivr_month; 

'''

### MAU per Month
매달 MAU를 계산하고, 지난 MAU를 병행 표기하겠다.

In [10]:
sql = '''

WITH mau AS (
SELECT
    DATE_TRUNC('month', order_date) :: DATE AS delivr_month,
    COUNT(DISTINCT user_id) AS mau
FROM orders
GROUP BY delivr_month)


SELECT
    delivr_month,
    mau,
    COALESCE(LAG(mau) OVER (ORDER BY delivr_month), 0) AS last_mau
FROM mau
ORDER BY delivr_month ASC;

'''

### Monthly MAU Growth Rate
이번달 MAU 와 지난달 MAU의 차이를 구하겠다.
또한 이번달 MAU가 지난달 MAU에 비해 몇% 성장했는지 구하겟다.

In [11]:
sql = '''

WITH mau AS (
SELECT
    DATE_TRUNC('month', order_date) :: DATE AS delivr_month,
    COUNT(DISTINCT user_id) AS mau
FROM orders
GROUP BY delivr_month),

mau_with_lag AS (
SELECT
    delivr_month,
    mau,
    COALESCE(LAG(mau) OVER (ORDER BY delivr_month),0) AS last_mau
FROM mau)



SELECT
    delivr_month,
    mau - last_mau AS mau_delta,
    ROUND(mau - last_mau::NUMERIC / last_mau, 2) AS growth
FROM mau_with_lag
ORDER BY delivr_month ASC;

'''

### Order Growth Rate
지난달 order수에 비해 이번달 order수가 얼만큼 성장했는지 구하겠다.

In [12]:
sql = '''

WITH orders AS (
SELECT
    DATE_TRUNC('month', order_date) :: DATE AS delivr_month,
    COUNT(DISTINCT order_id) AS orders
FROM orders
GROUP BY delivr_month),

orders_with_lag AS (
SELECT
    delivr_month,
    orders,
    COALESCE(LAG(orders) OVER (ORDER BY delivr_month ASC),1) AS last_orders
FROM orders)



SELECT
    delivr_month,
    ROUND((orders - last_orders) :: NUMERIC / last_orders,2) AS growth
    FROM orders_with_lag
ORDER BY delivr_month ASC;

'''

### Retention Rate

지난달의 이용자 수에 비해 이번달 이용자 수가 얼만큼 유지했는지 구하겠다.

In [13]:
sql = '''

WITH user_monthly_activity AS (
SELECT DISTINCT
    DATE_TRUNC('month', order_date) :: DATE AS delivr_month,
    user_id
FROM orders)



SELECT
    previous.delivr_month,
    ROUND(COUNT(DISTINCT current.user_id) :: NUMERIC /GREATEST(COUNT(DISTINCT previous.user_id), 1), 2) AS retention_rate
FROM user_monthly_activity AS previous
LEFT JOIN user_monthly_activity AS current
ON previous.user_id = current.user_id
AND previous.delivr_month = (current.delivr_month - INTERVAL '1 month')
GROUP BY previous.delivr_month
ORDER BY previous.delivr_month ASC;

'''

### Average Revenue per User

각각 user에 대한 평균 revenue를 구하겠다.

In [14]:
sql = '''

SELECT
    o.user_id,
    SUM(m.meal_price * o.order_quantity) AS revenue
FROM meals AS m
JOIN orders AS o ON m.meal_id = o.meal_id
GROUP BY user_id;

'''

### ARPU per Week
매주 ARPU를 구하겠다.

In [15]:
sql = '''

WITH kpi AS (
SELECT
    DATE_TRUNC('week', o.order_date) :: DATE AS delivr_week,
    SUM(m.meal_price * o.order_quantity) AS revenue,
    COUNT(DISTINCT o.user_id) AS users
FROM meals AS m
JOIN orders AS o ON m.meal_id = o.meal_id
GROUP BY delivr_week)

SELECT
delivr_week,
ROUND(revenue :: NUMERIC / GREATEST(users, 1),2) AS arpu
FROM kpi
ORDER BY delivr_week ASC;

'''

### Average Orders per User
평균 order 수를 구하겠다.

In [16]:
sql = '''

WITH kpi AS (
SELECT
    COUNT(DISTINCT order_id) AS orders,
    COUNT(DISTINCT user_id) AS users
FROM orders)

SELECT
  ROUND(orders :: NUMERIC / GREATEST(users, 1),2) AS arpu
FROM kpi;

'''

### Histogram of revenue
revenue 100 단위로 user의 수가 얼마나 분포되어있는지 구하겠다.

In [17]:
sql = '''

WITH user_revenues AS (
SELECT
    o.user_id,
    SUM(m.meal_price * o.order_quantity) AS revenue
FROM meals AS m
JOIN orders AS o ON m.meal_id = o.meal_id
GROUP BY user_id)



SELECT
    ROUND(revenue :: NUMERIC, -2) AS revenue_100,
    COUNT(DISTINCT user_id) AS users
FROM user_revenues
GROUP BY revenue_100
ORDER BY revenue_100 ASC;

'''

### Bucketing Users by Revenue

- `revenue < 150` 일때는 `Low-revenue users`
- `revenue < 300` 일때는 `Mid-revenue users`
- 그 이상일때는 `High-revenue users` 로 구분하고,
각각의 범주에 따라서 user의 수를 구하겠다.

In [18]:
sql = '''

WITH user_revenues AS (
SELECT
    user_id,
    SUM(m.meal_price * o.order_quantity) AS revenue
FROM meals AS m
JOIN orders AS o ON m.meal_id = o.meal_id
GROUP BY user_id)

SELECT
  CASE
    WHEN revenue < 150 THEN 'Low-revenue users'
    WHEN revenue < 300 THEN 'Mid-revenue users'
    ELSE 'High-revenue users'
  END AS revenue_group,
  COUNT(DISTINCT user_id) AS users
FROM user_revenues
GROUP BY revenue_group;

'''

### Bucketing Users by Orders

- `order < 8` 일때는 `Low-orders users`
- `order < 15` 일때는 `Mid-orders users`
- 그 이상일때는 `High -orders users` 로 구분하고, 각각의 범주에 따라서 user의 수를 구하겠다.

In [19]:
sql = '''

WITH user_orders AS (
SELECT
    user_id,
    COUNT(DISTINCT order_id) AS orders
FROM orders
GROUP BY user_id)



SELECT
CASE
    WHEN orders < 8 THEN 'Low-orders users'
    WHEN orders < 15 THEN 'Mid-orders users'
    ELSE 'High-orders users'
END AS order_group,
    COUNT(DISTINCT user_id) AS users
FROM user_orders
GROUP BY order_group;

'''

### Revenue Quartiles
Revenue의 25%구간, 50%구간, 75%구간을 구하고, 평균 revenue를 구한다.

In [20]:
sql = '''

WITH user_revenues AS (
SELECT
    user_id,
    SUM(m.meal_price * o.order_quantity) AS revenue
FROM meals AS m
JOIN orders AS o ON m.meal_id = o.meal_id
GROUP BY user_id)



SELECT
  ROUND(
    PERCENTILE_CONT(0.25) WITHIN GROUP
    (ORDER BY revenue ASC) :: NUMERIC,
  2) AS revenue_p25,
  ROUND(
    PERCENTILE_CONT(0.5) WITHIN GROUP
    (ORDER BY revenue ASC) :: NUMERIC,
  2) AS revenue_p50,
  ROUND(
    PERCENTILE_CONT(0.75) WITHIN GROUP
    (ORDER BY revenue ASC) :: NUMERIC,
  2) AS revenue_p75,
  ROUND(AVG(revenue) :: NUMERIC, 2) AS avg_revenue
FROM user_revenues;

'''

### Rank User by their Count of Orders

In [21]:
sql = '''
SELECT
    user_id,
    COUNT(DISTINCT order_id) AS count_orders
FROM orders
WHERE DATE_TRUNC('month', order_date) = '2018-08-01'
GROUP BY user_id;

'''

### Pivoting User Revenues by Month

각각의 user가 6월 7월 8월에 얼마나 Revenues를 창출했는지 구하겠다.

In [22]:
sql = '''

CREATE EXTENSION IF NOT EXISTS tablefunc;

SELECT * FROM CROSSTAB($$
    SELECT
        o.user_id,
        DATE_TRUNC('month', o.order_date) :: DATE AS delivr_month,
        SUM(m.meal_price * o.order_quantity) :: FLOAT AS revenue
      FROM meals AS m
      JOIN orders AS o ON m.meal_id = o.meal_id
     WHERE o.user_id IN (0, 1, 2, 3, 4) AND o.order_date < '2018-09-01'
     GROUP BY o.user_id, delivr_month
     ORDER BY o.user_id, delivr_month;
$$)
AS ct (user_id INT,
       "2018-06-01" FLOAT,
       "2018-07-01" FLOAT,
       "2018-08-01" FLOAT)
ORDER BY user_id ASC;

'''